In [27]:
import pandas as pd
from prophet import Prophet
from pymongo import MongoClient
from prophet.plot import plot_plotly, plot_components_plotly

db = MongoClient('localhost', 27017)
collection = 'covid_county_formatted'
cursor = db.sustaindb[collection].find()
df = pd.DataFrame(list(cursor))
df

,_id,date,county,state,fips,cumulative_cases,cumulative_deaths,cases,deaths,GISJOIN,formatted_date
0,605420df8dd202bf891eb049,2020-03-25,Autauga,Alabama,01001,4,0.0,3,0,G0100010,2020-03-25
1,605420df8dd202bf891eb04a,2020-03-28,Autauga,Alabama,01001,6,0.0,0,0,G0100010,2020-03-28
2,605420df8dd202bf891eb04b,2020-03-29,Autauga,Alabama,01001,6,0.0,0,0,G0100010,2020-03-29
3,605420df8dd202bf891eb04c,2020-03-30,Autauga,Alabama,01001,7,0.0,1,0,G0100010,2020-03-30
4,605420df8dd202bf891eb04d,2020-03-31,Autauga,Alabama,01001,7,0.0,0,0,G0100010,2020-03-31
...,...,...,...,...,...,...,...,...,...,...,...
1064945,605420f18dd202bf892ee89b,2020-08-30,Washakie,Wyoming,56043,107,5.0,0,0,G5600430,2020-08-30
1064946,605420f18dd202bf892ee89c,2020-09-04,Washakie,Wyoming,56043,109,6.0,1,0,G5600430,2020-09-04
1064947,605420f18dd202bf892ee89d,2020-08-31,Washakie,Wyoming,56043,107,5.0,0,0,G5600430,2020-08-31
1064948,605420f18dd202bf892ee89e,2020-08-29,Washakie,Wyoming,56043,107,5.0,0,0,G5600430,2020-08-29


In [48]:
df = df[["date", "GISJOIN", "cases", "deaths"]]

all_gis_joins = df['GISJOIN'].unique()
print(f'#GISJOINs: {len(all_gis_joins)}')

df0 = df.loc[df['GISJOIN'] == all_gis_joins[0]]
df0 = df0.rename(columns = {
    'date': 'ds',
    'cases': 'y'
})
df0

#GISJOINs: 3116


,ds,GISJOIN,y,deaths
0,2020-03-25,G0100010,3,0
1,2020-03-28,G0100010,0,0
2,2020-03-29,G0100010,0,0
3,2020-03-30,G0100010,1,0
4,2020-03-31,G0100010,0,0
...,...,...,...,...
3150,2020-09-07,G0100010,6,0
3151,2020-09-10,G0100010,22,0
3152,2020-09-11,G0100010,7,0
3153,2020-09-12,G0100010,14,0


In [49]:
model = Prophet(
    interval_width=0.95,
    growth='linear',
    daily_seasonality=False,
    weekly_seasonality=True,
    yearly_seasonality=True,
    seasonality_mode='multiplicative'
)

# model.fit(df0)
m = Prophet()
m.fit(df0)
df0_future = m.make_future_dataframe(periods=365)
df0_forecast = m.predict(df0_future)
df0_forecast

df0_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

INFO:prophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat,yhat_lower,yhat_upper
0,2020-03-24,5.928523,-13.826868,24.950116
1,2020-03-25,2.538198,-16.387674,22.758285
2,2020-03-26,4.551130,-15.462555,23.495693
3,2020-03-27,2.084051,-19.189924,21.845346
4,2020-03-28,0.476235,-19.121344,19.919808
...,...,...,...,...
709,2022-03-03,-45.358355,-102.597465,9.077196
710,2022-03-04,-48.100980,-104.250275,4.043099
711,2022-03-05,-49.984343,-102.835298,3.868638
712,2022-03-06,-49.708821,-102.459656,4.882562


In [26]:
# m.plot_components(df0_forecast)

In [25]:
# import plotly.graph_objs as go
# plot_plotly(m, df0_forecast)

In [31]:
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import *


result_schema = StructType([
    StructField("ds", DateType(), True),
    StructField("yhat", DoubleType(), True),
    StructField("yhat_lower", DoubleType(), True),
    StructField("yhat_upper", DoubleType(), True)
])

@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def forecast_store_item(history_pd):

    # instantiate the model, configure the parameters
    model = Prophet()

    # fit the model
    model.fit(history_pd)

    # configure predictions
    future_pd = model.make_future_dataframe(
        periods=90,
        freq='d',
        include_history=True
    )

    # make predictions
    results_pd = model.predict(future_pd)

    # . . .

    # return predictions
    return results_pd

In [32]:
result_schema = StructType([
    StructField("ds", DateType(), True),
    StructField("yhat", DoubleType(), True),
    StructField("yhat_lower", DoubleType(), True),
    StructField("yhat_upper", DoubleType(), True)
])

@pandas_udf(result_schema, PandasUDFType.GROUPED_MAP)
def temp(history_pd):

    # instantiate the model, configure the parameters
    m = Prophet()
    m.fit(df0)
    df0_future = m.make_future_dataframe(periods=365)
    df0_forecast = m.predict(df0_future)

    return df0_forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]


In [44]:
# df0 = df0[["ds", "y"]]
# df0.set_index('ds')
# temp(df0[["ds", "GISJOIN", "y", "deaths"]])
# df0

# results = (
#     df.groupBy('GISJOIN')
#     .apply(temp)
# )
# df0_ = df0.withColumn('GISJOIN', lit(None))

# from pyspark.sql import SparkSession

# spark = SparkSession \
#     .builder\
#     .master('local')\
#     .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/sustaindb.covid_county_formatted')\
#     .getOrCreate()

# spark_df = spark.read.format("mongo").option("uri",
#     "mongodb://localhost/sustaindb.covid_county_formatted").load()
# spark_df
#     .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.12:3.0.1.jar')\

# spark.sparkContext.addPyFile('./jars/mongo-spark-connector_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/spark-core_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/spark-sql_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/spark-mllib_2.12-3.0.1.jar')
# spark.sparkContext.addPyFile('./jars/bson-4.0.5.jar')
# spark.sparkContext.addPyFile('./jars/mongo-java-driver-3.12.5.jar')

# df1 = spark.read.format('com.mongodb.spark.sql.DefaultSource')\
#             .option('database', 'sustaindb')\
#             .option('collection', 'covid_county_formatted')\
#             .load()

df = spark.read.format('json').load('./covid_county.json')

In [45]:
df = df.select('GISJOIN', 'cases', 'deaths', 'date', 'formatted_date')
df.show()

+--------+-----+------+----------+--------------------+
| GISJOIN|cases|deaths|      date|      formatted_date|
+--------+-----+------+----------+--------------------+
|G0100010|    3|     0|2020-03-25|[2020-03-25T00:00...|
|G0100010|    0|     0|2020-03-28|[2020-03-28T00:00...|
|G0100010|    0|     0|2020-03-29|[2020-03-29T00:00...|
|G0100010|    1|     0|2020-03-30|[2020-03-30T00:00...|
|G0100010|    0|     0|2020-03-31|[2020-03-31T00:00...|
|G0100010|    3|     0|2020-04-01|[2020-04-01T00:00...|
|G0100010|    0|     0|2020-04-04|[2020-04-04T00:00...|
|G0100010|    0|     0|2020-04-05|[2020-04-05T00:00...|
|G0100010|    0|     1|2020-04-06|[2020-04-06T00:00...|
|G0100010|    0|     0|2020-04-02|[2020-04-02T00:00...|
|G0100010|    0|     0|2020-04-07|[2020-04-07T00:00...|
|G0100010|    0|     0|2020-04-08|[2020-04-08T00:00...|
|G0100010|    5|     0|2020-04-09|[2020-04-09T00:00...|
|G0100010|    0|     0|2020-04-10|[2020-04-10T00:00...|
|G0100010|    2|     0|2020-04-11|[2020-04-11T00

In [46]:
# df0 = df0.rename(columns = {
#     'date': 'ds',
#     'cases': 'y'
# })
df_cases = df.select('GISJOIN', 'date', 'cases').withColumnRenamed('date', 'ds').withColumnRenamed('cases', 'y')
# df_cases.show()

results = (
    df_cases.groupBy('GISJOIN')
    .apply(temp)
)
results.toPandas()

Py4JJavaError: An error occurred while calling o530.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 25.0 failed 1 times, most recent failure: Lost task 0.0 in stage 25.0 (TID 44, 10.253.202.195, executor driver): java.lang.UnsupportedOperationException: sun.misc.Unsafe or java.nio.DirectByteBuffer.<init>(long, int) not available
	at io.netty.util.internal.PlatformDependent.directBuffer(PlatformDependent.java:490)
	at io.netty.buffer.NettyArrowBuf.getDirectBuffer(NettyArrowBuf.java:243)
	at io.netty.buffer.NettyArrowBuf.nioBuffer(NettyArrowBuf.java:233)
	at io.netty.buffer.ArrowBuf.nioBuffer(ArrowBuf.java:245)
	at org.apache.arrow.vector.ipc.message.ArrowRecordBatch.computeBodyLength(ArrowRecordBatch.java:222)
	at org.apache.arrow.vector.ipc.message.MessageSerializer.serialize(MessageSerializer.java:240)
	at org.apache.arrow.vector.ipc.ArrowWriter.writeRecordBatch(ArrowWriter.java:132)
	at org.apache.arrow.vector.ipc.ArrowWriter.writ

In [23]:
 from pyspark.sql import SparkSession 

 spark = SparkSession.builder.master("local").getOrCreate() 
 print(spark.sparkContext.version)

3.0.1
